<a href="https://colab.research.google.com/github/arind123/GenAI/blob/main/5-NewsClassification_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip install textblob 'keras-nlp' 'keras-preprocessing' 'gensim==4.2.0' np_utils swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.5/584.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 81.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 104.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.0 MB/s eta 0:00:00
  Created wheel for np_utils

In [3]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda, ELU, Conv1D, MaxPooling1D, Dropout
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk
import pickle
from tensorflow.nn import leaky_relu

import re
import warnings
from sklearn.model_selection import train_test_split
from textblob import TextBlob

TRACE = False
embedding_dim = 300
epochs=100
batch_size = 250
corpus_size=25000
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
%%writefile get_data.sh
if [ ! -f news.csv ]; then
  wget -O news.csv https://www.dropbox.com/s/352x7xzivf60zgc/news.csv?dl=0
fi

Writing get_data.sh


In [5]:
!bash get_data.sh

--2023-11-21 13:58:57--  https://www.dropbox.com/s/352x7xzivf60zgc/news.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/352x7xzivf60zgc/news.csv [following]
--2023-11-21 13:58:57--  https://www.dropbox.com/s/raw/352x7xzivf60zgc/news.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc807efaf581ba81d60c8a9e53b0.dl.dropboxusercontent.com/cd/0/inline/CH_ihR9pVcK0H640Z6e2W4jBTzhKnudB4W4DRt2G-e-MN22Jv3-i-VUnQMnrany_zfNSOSJCZfQLHXQG1GdHblaHzHIxQv5l59ojK4GabAq9Ce8klI2y8uQ1rOVUJKoHdn59R6iC1oSjXI643i0JzMDQ/file# [following]
--2023-11-21 13:58:58--  https://uc807efaf581ba81d60c8a9e53b0.dl.dropboxusercontent.com/cd/0/inline/CH_ihR9pVcK0H640Z6e2W4jBTzhKnudB4W4DRt2G-e-MN22Jv3-i-VUnQMnrany_zfNSOSJCZfQLHXQG1GdHblaHzHIxQv5l59ojK4GabAq9C

In [6]:
path = './news.csv'
news_pre = pd.read_csv(path, header=0).sample(n=corpus_size).reset_index(drop=True)

In [7]:
news_pre.head()

,category,title
0,Business,"BBC set for major shake-up, claims newspaper"
1,Business,Marsh averts cash crunch
2,Sports,"Jeter, Yankees Look to Take Control (AP)"
3,Sci/Tech,Flying the Sun to Safety
4,Business,Stocks Seen Flat as Nortel and Oil Weigh


In [8]:
news_pre['category'].value_counts()

Sci/Tech    6329
Sports      6303
World       6215
Business    6153
Name: category, dtype: int64

In [9]:
# Define X and y.
X = news_pre.title
y = news_pre.category

# Split the new DataFrame into training and testing sets.
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [10]:
# def preprocess_text(text, should_join=True):
#     # Use the tokenizer to tokenize into words, lowercase them, remove punctuation, and finally use gensim.utils.simple_preprocess(text)
#     text = None
#     if should_join:
#       return ' '.join(gensim.utils.simple_preprocess(text))
#     else:
#       return gensim.utils.simple_preprocess(text)

import re
nltk.download('stopwords')
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'


def preprocess_text(text, should_join=True):
    text = ' '.join(word.lower() for word in textblob_tokenizer(text))
    # text = re.sub(r'http\S+', '', text) # remove http links
    # text = re.sub(r'bit.ly/\S+', '', text) # rempve bitly links
    # text = text.strip('[link]') # remove [links]
    text = re.sub('['+my_punctuation + ']+', ' ', text) # remove punctuation
    text = re.sub('\s+', ' ', text) #remove double spacing
    # text = re.sub(r"[^a-zA-Z.,&!?]+", r" ", text) # only normal characters
    # text_token_list = [word for word in text.split(' ')
    #                         if word not in my_stopwords] # remove stopwords
    # text_token_list = [word_rooter(word) if '#' not in word else word
    #                     for word in text_token_list] # apply word rooter
    # text = ' '.join(text_token_list)
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
import swifter
# Use swifter to apply the preprocessin and save that pandas series to a file
news = X.swifter.apply(preprocess_text)

Pandas Apply:   0%|          | 0/25000 [00:00<?, ?it/s]

In [12]:
news.to_csv("./news_processed.csv",index=False)

In [13]:

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = 'news_processed.csv'
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield preprocess_text(line, should_join=False)


# word2vec_model = None
# Get a word2vec model using gensim.models and passing the sentences using MyCorpus()
import gensim.models

sentences = MyCorpus()
word2vec_model = gensim.models.Word2Vec(sentences=sentences,
                                        vector_size = embedding_dim)

In [14]:
weights = word2vec_model.wv.vectors  # Get the weights of the model (the embedding) and convert to tensor. Hint: Check word2vec_model.wv
vocab_size = len(word2vec_model.wv.index_to_key)  # get vocab size from index_to_key in word2vec_model.wv

In [15]:
weights.shape

(4929, 300)

In [16]:
news_preprocessed = pd.DataFrame()
news_preprocessed['label'] = news_pre.category.map({'Business': 0, 'Sports': 1, 'Sci/Tech': 2, 'World': 3})
news_preprocessed['title'] = news
news_preprocessed

,label,title
0,0,bbc set for major shake up claims newspaper
1,0,marsh averts cash crunch
2,1,jeter yankees look to take control ap
3,2,flying the sun to safety
4,0,stocks seen flat as nortel and oil weigh
...,...,...
24995,0,wal mart clarifies policy regarding unions ap
24996,2,viewpoint better russian planning would preven...
24997,1,no syracuse princeton
24998,1,usc oklahoma still auburn third in bcs


In [17]:
def get_maximum_review_length(df):
    maximum = 0
    for ix, row in df.iterrows():
        candidate = len(textblob_tokenizer(row.title))
        if candidate > maximum:
            maximum = candidate
    return maximum


maximum = get_maximum_review_length(news_preprocessed)   # Since 2 titles may have different number of words, we have to find the max length and fill with 0s if a title is shorter

In [26]:
X = np.zeros((len(news), maximum))
X.shape

(25000, 18)

In [19]:
# Here we do what we said above
# Iterate through the news df and for every word, if it exists in the word2vec model, put into X for that review and that word the index of the embedding (check index_to_key)
# HINT: to iterate through a column of a pandas dataframe you do:

# for news_txt in news:
#   words_list = textblob_tokenizer(news_txt)
#   for _words in words_list:
#     if _words in word2vec_model.wv.index_to_key:

for index, val in news_pre.iterrows():
  words_list = preprocess_text(val[1], False)
  for word in words_list:
    if word in word2vec_model.wv.index_to_key:
      X[index, words_list.index(word)] = word2vec_model.wv.index_to_key.index(word)
    # else:
    #   X[index, words_list.index(word)] = vocab_size + 1

# FILL
y = news_preprocessed.label

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = tf.constant(X_train)
X_test = tf.constant(X_test)
# Convert y_train and y_test from an array of values between 0-3 to a one hot matrix tensor
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)


In [49]:
window = 2
model = Sequential()
model.add(Embedding(input_dim=weights[0],
                    output_dim=weights[1],
                    input_length=window*2,
                    embeddings_initializer = tf.keras.initializers.Constant(weights),
                    trainable = False,
                    mask_zero = True))  # Add an Embedding layer with weights being the rweights variable and trainable as False. The embedding dimension should be embedding_dim
model.add(Dense(90, activation=tf.keras.layers.LeakyReLU(alpha=0.01)))  # Add a couple of Dense Layers with RELU or leaky_relu activations. You may add Batch Norm if you want too
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(None, embedding_dim,)))  # Average out the words of the sentence. The expected out is (N, D) where N is number f samples in batch and D is embedding dimension
model.add(Dense(embedding_dim, activation='sigmoid'))  # Add final Dense layer

In [50]:
# Compile the model. Think what is the best loss to use
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 4, 300)            1478700   
                                                                 
 dense_13 (Dense)            (None, 4, 90)             27090     
                                                                 
 lambda_6 (Lambda)           (None, 90)                0         
                                                                 
 dense_14 (Dense)            (None, 300)               27300     
                                                                 
Total params: 1533090 (5.85 MB)
Trainable params: 54390 (212.46 KB)
Non-trainable params: 1478700 (5.64 MB)
_________________________________________________________________


In [51]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.01)
# history = None # Fit the model, use the callback above to do EarlyStopping

model.fit(X_train, y_train,epochs=10, verbose=1, callbacks = [callback])

Epoch 1/10


ValueError: ignored

In [ ]:
import matplotlib.pyplot as plt

# function for plotting loss
def plot_metrics(train_metric, val_metric=None, metric_name=None, title=None, ylim=5):
    plt.title(title)
    plt.ylim(0,ylim)
    plt.plot(train_metric,color='blue',label=metric_name)
    if val_metric is not None: plt.plot(val_metric,color='green',label='val_' + metric_name)
    plt.legend(loc="upper right")

# plot loss history
plot_metrics(history.history['loss'], history.history['val_loss'], "Loss", "Loss", ylim=2.0)


In [ ]:
# Test with the following two sentences:
# - 'supercomputer will put workers jobless soon'
# - 'patriots goes winning super bowl'

x_val = np.zeros((2, maximum))
for index, row in enumerate(['supercomputer will put workers jobless soon', 'patriots goes winning super bowl']):
    pass
y_val = tf.one_hot([0,1], depth=4)